# Design Assist: PLL Stability

## Transfer function

Let us consider the closed-loop PLL response, $H(s)$, built from the open-loop characteristics $G(s)$:
$$ H(s)\triangleq\frac{\varphi_{out}(s)}{\varphi_{in}(s)}=\frac{G(s)}{1+G(s)}$$

where the open-loop response is given by:
$$ G(s)=\frac{K}{s}\left(\frac{1+s/\omega_z}{1+s/\omega_p}\right).$$

The 2nd order closed-loop response can therefore be expanded as:
$$H(s)=K\omega_p\frac{1+s/\omega_z}{s^2+s\omega_p(1+K/\omega_z) + K\omega_p}.$$

The above equation can also be re-written in standard form as:
$$H(s)=\omega_n^2\frac{1+s(2\zeta/\omega_n)}{s^2+2\zeta\omega_n s + \omega_n^2}$$
where
$$\omega_n^2=K\omega_p$$
$$\zeta=\frac{1}{2}\left(\frac{\omega_p}{\omega_n}+\frac{\omega_n}{\omega_z}\right)$$

## Step Error Response

$$Y_{u,error}=
\frac{1}{2}\left(\zeta\frac{2\alpha-1}{\sqrt{\zeta^2-1}}+1\right)e^{-\omega_n t(\zeta+\sqrt{\zeta^2-1})}
-\frac{1}{2}\left(\zeta\frac{2\alpha-1}{\sqrt{\zeta^2-1}}-1\right)e^{-\omega_n t(\zeta-\sqrt{\zeta^2-1})}
$$
where
$$\alpha \triangleq \frac{\omega_n}{2\zeta K}$$

## References

 1. W. Egan, Phase-Lock Basics, 2nd ed., Wiley-Interscience, 2007, ISBN: 978-0-470-11800-9.

In [ ]:
include("PhaseLock.jl")
import PhaseLock: Xf1, Sys2
import InspectDR
using NumericIO

#Convenience aliases:
SI(x) = formatted(x, :SI, ndigits=3)

#When MIME"image/svg+xml" is disabled, Jupyter eventually requests PNG inline graphics:
#(SVG outputs do not render well in notebooks for some reason...)
InspectDR.defaults.rendersvg = false

nothing

# Setup: Interact, Reactive, and Signals
(Also create initial plot object)

In [ ]:
using Interact, Reactive

#fopt = [("kHz", 1.0e3), ("MHz", 1.0e6), ("GHz", 1.0e9)] #Make sure to use Float64
#freqmap = Dict{Float64, String}(v=>s for (s,v) in fopt)
#fmult = Signal(Float64, 1e9) #Frequency multiplier
#annot = Signal(Bool, true) #Annotate plot
fmin = Signal(Float64, 1e5) #Minimum plot frequency
fmax = Signal(Float64, 1e9) #Maximum plot frequency
tmax = Signal(Float64, 10e-6) #Maximum plot time
ωnperiods = Signal(Float64, 10) #Maximum plot time (relative to ωn)
G = Signal(PhaseLock.Xf1(K=10.0^(150/20), ωp=2pi*500e3, ωz=2pi*50e6)) #Open-loop characteristics
pobj = PhaseLock.newplot() #Create initial plot object

nothing

# Interact/Reactive Control: Plot limits


## TODO

In [ ]:
#txt_fmax = textbox(value(fmax), label="max freq", signal=fmax) #Will not display - notebook hangs??
#display(txt_fmin) Will hang session using this method, for some reason
txt_fmin = textbox(value(fmin), label="min freq") #This one does not cause hangups
fmin = signal(txt_fmin) #Hack to get fmax/txt_fmax working
txt_fmax = textbox(value(fmax), label="max freq")
fmax = signal(txt_fmax)
txt_tmax = textbox(value(tmax), label="max time")
tmax = signal(txt_tmax)
txt_ωnperiods = textbox(value(ωnperiods), label="ωn periods")
ωnperiods = signal(txt_ωnperiods)
map(display, [txt_fmin, txt_fmax, txt_tmax, txt_ωnperiods])
nothing

# Interact/Reactive Control: Inline Plots
Use sliders/text boxes to change system parameters

## TODO

- Figure out why Interact does not update HTML correctly when using \$LaTeX\$

In [ ]:
#Gain slider:
sld_KdB = slider(-10:1.0:400, value=20*log10(value(G).K), label="OL gain (dB)")
display(sld_KdB)

#Pole/zero sliders (log(f) values):
pzrange = 1:0.1:9
sld_fp = slider(1:0.1:9, value=log10(value(G).ωp/(2pi)), label="log(pole)")
sld_fz = slider(1:0.1:9, value=log10(value(G).ωz/(2pi)), label="log(zero)")
map(display, [sld_fp, sld_fz])

#Build widget list to control plot for future code blocks
#TODO: Why do widgets have to be displayed in stages?
#widgetlist = [tgl_fmult, chk_annot, txt_fmax, sld_G, sld_fz, sld_fp1, sld_fp2]

#Compute open-loop characteristics
G_calc = map(signal(sld_KdB), signal(sld_fp), signal(sld_fz)) do _KdB, lfp, lfz
    push!(G, PhaseLock.Xf1(K=10.0^(_KdB/20), ωp=2pi*10^lfp, ωz=2pi*10^lfz))
end

info_poles = map(G) do _G
    txt(x::String) = "$x" #"\\textrm{$x}" #Not currently using LaTeX
    return HTML(txt("G (open-loop): ") *
        "K=" * SI(20*log10(_G.K)) * txt(" dB") *
        ", 𝑓p=" * txt(SI(_G.ωp/(2pi)) * "Hz") *
        ", 𝑓z=" * txt(SI(_G.ωz/(2pi)) * "Hz") *
        txt(" (") * "𝑓0=" * txt(SI(PhaseLock.ω_0(_G)/(2pi))) * "Hz)"
    )
end
display(info_poles)

info_sys = map(G) do _G
    s = PhaseLock.Sys2(_G)
    α = SI(s.α); ζ = SI(s.ζ); ωn = SI(s.ωn)
    return HTML("H (closed-loop): α=$α, ζ=$ζ, <b>ωn=$(ωn)rad/s</b>")
end
display(info_sys)

#Widget-controlled plot:
updater_plot = map(fmin, fmax, tmax, ωnperiods, G) do _fmin, _fmax, _tmax, _ωnperiods,_G
    s = PhaseLock.Sys2(_G)
    PhaseLock.update(pobj, _fmin, _fmax, _tmax, _ωnperiods, _G)
end
display(updater_plot)

nothing

# Interact/Reactive Control: Inspect/Gtk GUI

In [ ]:
gtkgui = display(InspectDR.GtkDisplay(), pobj) #Display plot in Gtk GUI.

#Refresh GUI when plot changes:
updater_gtkgui = map((p)->InspectDR.refresh(gtkgui), updater_plot)

#Re-display GUI controls, for convenience:
#map(display, widgetlist)

nothing

# Demo complete!